# Anomaly-free theories with $T=0$

This notebook explores the accompanying data of anomaly-free theories with no tensor multiplets, exemplifying the "helper" functions in `helper.py`.

The data files consist of the following:
  - **Irreps:**

    Information about irreps (`[irrep-ID] [H] [A] [B] [C] [quaternionic] [highest-weight vector]`) is stored for each simple group in a separate file.
    Irrep IDs have the general structure `irr-[group-ID]-[dim R]`. If there are multiple irreps of the same dimension then a latin label has been added: for example for $\mathrm{SO}(8)$ `irr-D4-8a` is $\mathbf{\underline{8}_\mathrm{v}}$ while `irr-D4-8b` is $\mathbf{\underline{8}_\mathrm{s}}$. If the irrep is quanternionic and free of Witten anomalies then its ID has an additional tag `-h`, all of $H$, $A$, $B$ and $C$ have been divided by two and multiplicies always refer to number of half-hypermultiplets. For example, `irr-E7-56-h` has $H=28$ and `3 x (irr-E7-56-h)` should be interpreted as $\frac{3}{2}\times\mathbf{\underline{56}}$.

  - **Vertices:**

    Vertices (`[vertex-ID] [Δ] [bi.bi] [b0.bi] [hypers]`) are similarly stored separated for each simple group. Vertex IDs have the structure `vtx-[group-ID]-[Δ]-[bi.bi]-[b0.bi]`, plus an identifying latin label `-[latin]` if there are multiple vertices with identical group, $\Delta=H-V$, $b_i\cdot b_i$ and $b_0\cdot b_i$ but different hypers. Information about hypers is stored as `nR x (irrep-ID) + ...`, for example `9 x (irr-E6-27) + 1 x (irr-E6-78)`.

  - **Irreducible cliques:**

    Irreducible cliques (`[clique-ID] [vertex-IDs] [Δ] [Δ+28n] [T-min] [signature] [bi.bi] [b0.bi] [hypers]`) for $T=0$ are stored separately by the number of vertices, $k$. Clique IDs have the structure `clq-[numA/B]-[signature]-[Δ]-[hash]`, where `numA/B` is the number of type-A and type-B vertices (for $T=0$ there are only type-A vertices present), `signature` is the number of positive and negative eigenvalues of the $k\times k$ matrix $b_i\cdot b_j$ and `hash` is to ensure the ID is unique.

### Set-up

In [ ]:
import numpy as np
import glob
from src import helper

In [ ]:
folder = './data-T=0'   # For displaying/fetching irrep/vertex data from files
files = glob.glob('./data-T=0/cliques-anomaly-free/cliques_A=*.tsv')    # For loading in irreducible cliques

### Irreps
View irreps for a specified simple group with `helper.display_irreps()`.

In [ ]:
helper.display_irreps('C', 4, folder)

### Vertices

Vertex data can be loaded for a given simple group using `helper.get_vertex_data()`. The data are returned as a NumPy structured array with the fields `'ID'`, `'delta'` (for $\Delta=H-V$), `'bibi'`, `'b0bi'` and `'hypers'`. For each vertex, `hypers` itself is also a structured array with the fields `'nR'` and `'irrep'`.

In [ ]:
vertices = helper.get_vertex_data('A', 10, folder)
print(len(vertices))

print('\nID:', vertices['ID'])
print('delta:', vertices['delta'])
print('bibi:', vertices['bibi'])
print('b0bi:', vertices['b0bi'])

print('hypers:')
for vertex in vertices:
    print('  ', vertex['hypers'])

It is easy enough to find all vertices which contain a particular irrep:

In [ ]:
irrep = 'irr-A4-105'

group_ID = irrep.split('-')[1]
vertex_data = helper.get_vertex_data(group_ID[0], int(group_ID[1:]), folder)
contains_irrep = [irrep in vtx['hypers']['irrep'] for vtx in vertex_data]
vertices_w_irrep = vertex_data[contains_irrep]

print(f'{group_ID} vertices: {len(vertex_data)}')
print(f'with {irrep}: {len(vertices_w_irrep)}')

In [ ]:
for vertex in vertices_w_irrep:
    hypers_string = ' + '.join([f'{nR} x ({irr})' for nR, irr in vertex['hypers']])
    print(vertex['ID'], '\t', hypers_string)

### Cliques

Cliques can be loaded from a given file using `helper.get_irreducible_cliques()`. For $T=0$ there are few enough cliques that all of them can be loaded in simultaneously.

In [ ]:
cliques = [helper.get_irreducible_cliques(file) for file in files]
print([len(aa) for aa in cliques])

cliques = [aa for bb in cliques for aa in bb]
print(len(cliques))

All information about an irreducible clique can be viewed by calling `display()`.

In [ ]:
clq = np.random.choice(cliques)
clq.display()

Any list of cliques can be filtered to those containing particular vertices.

For example, using the vertices which contain a particular irrep from above:

In [ ]:
cliques_w_irrep = helper.filter_irreducible_cliques_by_vertex(cliques, vertices_w_irrep['ID'], 'any')
print(len(cliques_w_irrep))

In [ ]:
for clq in cliques_w_irrep:
    clq.display()

By switching `'any'` to `'all'` we can filter to cliques which contain *all* vertices (with multiplicity) from the supplied list of IDs:

In [ ]:
vertex_IDs = ['vtx-A03-149-4-6', 'vtx-D07-233-4-6']
# vertex_IDs = ['vtx-A03-233-16-12', 'vtx-A03-233-16-12']
# vertex_IDs = ['vtx-A03-83-1-3', 'vtx-A03-149-4-6', 'vtx-B03-156-4-6', 'vtx-C02-143-4-6']

cliques_w_vertices = helper.filter_irreducible_cliques_by_vertex(cliques, vertex_IDs, 'all')
print(len(cliques_w_vertices))

In [ ]:
for clq in cliques_w_vertices:
    clq.display()

Cliques can also be filtered to those containing a particular gauge group, either exactly...

In [ ]:
gauge_group = 'SU(5) x Sp(3) x SO(11)'
# gauge_group = 'SU(4) x SO(13)'
# gauge_group = 'F(4)'
# gauge_group = 'Sp(2) x Sp(2) x Sp(2) x Sp(2) x Sp(2) x Sp(2)'
# gauge_group = 'SU(7) x SU(7) x SU(7) x SU(7)'
# gauge_group = 'SU(24)'

cliques_w_group = helper.filter_irreducible_cliques_by_gauge_group(cliques, gauge_group, exact=True)
len(cliques_w_group)

In [ ]:
for clique in cliques_w_group:
    clique.display()

...or as a subgroup

In [ ]:
gauge_group = 'SU(7) x SU(7) x SU(7)'
# gauge_group = 'SU(4) x F(4)'
# gauge_group = 'SO(7) x G(2) x G(2)'

cliques_w_group = helper.filter_irreducible_cliques_by_gauge_group(cliques, gauge_group, exact=False)
len(cliques_w_group)

In [ ]:
for clique in cliques_w_group:
    clique.display()